In [1]:
from torch.nn import BCEWithLogitsLoss, BCELoss
from torch.optim import Adam
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
from imutils import paths
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import time
import os
import numpy as np

from Utils import loader, loss
from Train import train
from Model import Model

In [2]:
imdir_rs19 = '/home/physik/Dokumente/rs19/jpgs/rs19_val'
maskdir_rs19 = '/home/physik/Dokumente/rs19/uint8/rs19_val'
#imdir_rs19 = '/home/luis/Documents/ImSeg/rs19_val/jpgs/rs19_val'
#maskdir_rs19 = '/home/luis/Documents/ImSeg/rs19_val/uint8/rs19_val'
classes_ = {"rail-track"}
range_train = (0,2000)
range_test = (5000,6000)
range_val = (7,8)

In [3]:
traindata = loader.dataset(imdir=imdir_rs19,
                      maskdir=maskdir_rs19,
                      numstart=range_train[0],
                      numend=range_train[1],
                      classes=classes_)

testdata = loader.dataset(imdir=imdir_rs19,
                      maskdir=maskdir_rs19,
                      numstart=range_test[0],
                      numend=range_test[1],
                      classes=classes_)

valdata = loader.dataset(imdir=imdir_rs19,
                      maskdir=maskdir_rs19,
                      numstart=range_val[0],
                      numend=range_val[1],
                      classes=classes_)

In [4]:
traindata[0][1].shape

torch.Size([1, 512, 1024])

In [5]:
traindata[0][0].shape

torch.Size([3, 512, 1024])

In [6]:
traindata[0][1].dtype

torch.float32

In [7]:
len(traindata)

2001

In [8]:
len(testdata)

1001

In [9]:
batchsize = 1
#loss = torch.nn.BCEWithLogitsLoss()
lossf = loss.BCEDiceLoss()
optim = 'Adam'
lr = 0.00001
momentum = 0.01
decay_rate = 0.9
decay_steps =500
num_epochs = 10000
model_path = ""

In [10]:
model = Model.UNet(num_classes=1,enc_channels=(3,64,128,256,512),dec_channels=(512,256,128,64))

In [11]:
model = model.to('cuda')

In [12]:
Trainer = train.TrainSeg(model,
                         traindata,
                         testdata,
                         valdata,
                         classes_,
                         lossf,
                         optim,
                         batchsize,
                         lr,
                         momentum,
                         decay_rate,
                         decay_steps,
                         num_epochs,
                         model_path)

In [ ]:
Trainer.train()

Epoch: 1/10000
Train loss: 1.450788, Test loss: 1.2686, Learning rate: 0.000010
Epoch: 2/10000
Train loss: 1.279541, Test loss: 1.1560, Learning rate: 0.000010
Epoch: 3/10000
Train loss: 1.152583, Test loss: 1.0804, Learning rate: 0.000010


In [ ]:
Trainer.val()

In [ ]:
model.eval()

In [ ]:
loader.visualize_result(traindata[0][0],traindata[0][1][0,:,:])

In [ ]:
torch.max(traindata[0][1])

In [ ]:
result = torch.sigmoid(model(traindata[0][0].unsqueeze(0).to('cuda')))

In [ ]:
plt.imshow(result.detach().numpy()[0][0])

In [ ]:
result[result>=0.5]=1.

In [ ]:
loader.visualize_result(traindata[0][0],result[0][0])

In [ ]:
torch.min(result)

In [ ]:
torch.max(result)

In [ ]:
result.shape

In [ ]:
zeros = torch.zeros((1,1080,1920))

In [ ]:
loss(result[0],traindata[0][1])

In [ ]:
Trainer.model_path = "/home/physik/Model_28102022_1"

In [ ]:
Trainer.save()

In [ ]:
Trainer.load()

In [ ]:
loss.dice_score(traindata[0][1],traindata[0][1],None)

In [ ]:
loss.dice_score(result,traindata[0][1],None)

In [ ]:
import gc
gc.collect()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
for img,mask in Trainer.trainloader:
    print (img.shape)

In [ ]:
img = img.to('cuda')

In [ ]:
img.device

In [ ]:
del img

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainloader = DataLoader(traindata, batch_size = 2, shuffle = True, num_workers = 1)

In [ ]:
for (img,mask) in trainloader:
    print(mask.shape)